In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score


Read all dataset

In [5]:
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

Describe data

In [7]:
train_set.describe()
train_set[:2]

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000


In [26]:
train_set['user_id'].describe()
train_set['user_id'].unique()

array(['e00f8ff2eaf9', '39aeb48f0017', '91e2f88dd6e3', ...,
       'dd05217165ea', 'dc23ef451c8a', 'f3342ccd95c8'], dtype=object)

In [14]:
np.sum(train_set['user_id'] == '45ba3f23bf25')

1080

In [25]:
train_set['region'].describe()
train_set['region'].unique()

array(['Свердловская область', 'Самарская область', 'Ростовская область',
       'Татарстан', 'Волгоградская область', 'Нижегородская область',
       'Пермский край', 'Оренбургская область', 'Ханты-Мансийский АО',
       'Тюменская область', 'Башкортостан', 'Краснодарский край',
       'Новосибирская область', 'Омская область', 'Белгородская область',
       'Челябинская область', 'Воронежская область',
       'Кемеровская область', 'Саратовская область',
       'Владимирская область', 'Калининградская область',
       'Красноярский край', 'Ярославская область', 'Удмуртия',
       'Алтайский край', 'Иркутская область', 'Ставропольский край',
       'Тульская область'], dtype=object)

In [17]:
train_set[['parent_category_name', 'category_name']].describe()

,parent_category_name,category_name
count,1503424,1503424
unique,9,47
top,Личные вещи,"Одежда, обувь, аксессуары"
freq,697623,282753


In [23]:
train_set['parent_category_name'].value_counts()
train_set['parent_category_name'].unique()

array(['Личные вещи', 'Для дома и дачи', 'Бытовая электроника',
       'Транспорт', 'Недвижимость', 'Животные', 'Хобби и отдых', 'Услуги',
       'Для бизнеса'], dtype=object)

In [24]:
train_set['category_name'].value_counts()
train_set['category_name'].unique()

array(['Товары для детей и игрушки', 'Мебель и интерьер', 'Аудио и видео',
       'Автомобили', 'Ремонт и строительство',
       'Одежда, обувь, аксессуары', 'Детская одежда и обувь', 'Квартиры',
       'Товары для компьютера', 'Собаки', 'Дома, дачи, коттеджи',
       'Товары для животных', 'Другие животные', 'Комнаты',
       'Коллекционирование', 'Коммерческая недвижимость',
       'Посуда и товары для кухни', 'Красота и здоровье', 'Аквариум',
       'Телефоны', 'Часы и украшения', 'Предложение услуг', 'Птицы',
       'Спорт и отдых', 'Музыкальные инструменты', 'Бытовая техника',
       'Игры, приставки и программы', 'Земельные участки',
       'Продукты питания', 'Кошки', 'Билеты и путешествия',
       'Книги и журналы', 'Растения', 'Гаражи и машиноместа',
       'Мотоциклы и мототехника', 'Планшеты и электронные книги',
       'Оборудование для бизнеса', 'Настольные компьютеры', 'Ноутбуки',
       'Велосипеды', 'Грузовики и спецтехника', 'Готовый бизнес',
       'Фототехника', 'Вод

In [16]:
train_set

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796.0,0.80323
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823.0,0.00000
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567.0,0.80323
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415.0,0.00000
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46.0,0.00000


Data cleaning
1. change region to code
2. change city to code
3. category name
4. make mean and std of price to be 0 and 1 respectively

<b>Cleaning price</b>

In [56]:
clean_price = train_set['price'].fillna(0)
clean_price = preprocessing.scale(clean_price)
clean_price.mean()


2.776620948309052e-19

In [57]:
clean_price.std()

1.0000000000000002

<b>one hot encode user_type</b>

In [58]:
train_set['user_type'].unique()

array(['Private', 'Company', 'Shop'], dtype=object)

In [63]:
userTypes = pd.get_dummies(train_set['user_type'])


<b>encode city</b>

In [90]:
print(len(cities.unique()))
cities = train_set['city'].astype('category')
codedCities = cities.cat.rename_categories(np.arange(0, len(cities.unique()))).astype('float')
cleanCities = pd.Series(preprocessing.scale(codedCities))
cleanCities.describe()
len(cleanCities.unique())

1733


1733

In [71]:
train_set[1:3]

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


In [93]:
regions = train_set['region'].astype('category')
print(len(regions.unique()))
codedRegions = regions.cat.rename_categories(np.arange(0, len(regions.unique()))).astype('float')
cleanRegions = pd.Series(preprocessing.scale(codedRegions))
cleanRegions.describe()
print(len(cleanRegions.unique()))

28
28


In [94]:
categories = train_set['category_name'].astype('category')
print(len(categories.unique()))
codedCategories = categories.cat.rename_categories(np.arange(0, len(categories.unique()))).astype('float')
cleanCategories = pd.Series(preprocessing.scale(codedCategories))
cleanCategories.describe()
print(len(cleanCategories.unique()))

47
47


In [95]:
parentCategories = train_set['parent_category_name'].astype('category')
print(len(parentCategories.unique()))
codedParentCategories = parentCategories.cat.rename_categories(np.arange(0, len(parentCategories.unique()))).astype('float')
cleanParentCategories = pd.Series(preprocessing.scale(codedParentCategories))
cleanParentCategories.describe()
print(len(cleanParentCategories.unique()))

9
9


In [96]:
userIds = train_set['user_id'].astype('category')
print(len(userIds.unique()))
codeduserIds = userIds.cat.rename_categories(np.arange(0, len(userIds.unique()))).astype('float')
cleanuserIds = pd.Series(preprocessing.scale(codeduserIds))
cleanuserIds.describe()
print(len(cleanuserIds.unique()))

771769
771769


In [112]:
df = pd.DataFrame(cleanuserIds, columns=['userIds'])
df['region'] = cleanRegions
df['city'] = cleanCities
df['parent_category_name'] = cleanParentCategories
df['category_name'] = cleanCategories
df['price'] = clean_price
df['Company'] = userTypes['Company']
df['Private'] = userTypes['Private']
df['Shop'] = userTypes['Shop']
Y = train_set['deal_probability']
X = df
X.head()

,userIds,region,city,parent_category_name,category_name,price,Company,Private,Shop
0,1.302408,0.734012,-0.977106,0.101443,1.422787,-0.004592,0,1,0
1,-0.953026,0.466244,0.802000,-0.894372,-0.087887,-0.004552,0,1,0
2,0.242825,0.332360,0.751168,-1.890187,-1.598561,-0.004537,0,1,0
3,0.857857,1.001781,0.039526,0.101443,1.422787,-0.004564,1,0,0
4,1.509127,-1.274249,-1.279978,1.097258,-1.749629,-0.003983,0,1,0


RUn regressor

In [139]:
regressor = RandomForestRegressor(n_jobs=4,n_estimators=20)
regressor.fit(X, Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [140]:

# userIds
userIds = test_set['user_id'].astype('category')
print(len(userIds.unique()))
codeduserIds = userIds.cat.rename_categories(np.arange(0, len(userIds.unique()))).astype('float')
cleanuserIds = pd.Series(preprocessing.scale(codeduserIds))
cleanuserIds.describe()
print(len(cleanuserIds.unique()))

# regions
regions = test_set['region'].astype('category')
print(len(regions.unique()))
codedRegions = regions.cat.rename_categories(np.arange(0, len(regions.unique()))).astype('float')
cleanRegions = pd.Series(preprocessing.scale(codedRegions))
cleanRegions.describe()
print(len(cleanRegions.unique()))

categories = test_set['category_name'].astype('category')
print(len(categories.unique()))
codedCategories = categories.cat.rename_categories(np.arange(0, len(categories.unique()))).astype('float')
cleanCategories = pd.Series(preprocessing.scale(codedCategories))
cleanCategories.describe()
print(len(cleanCategories.unique()))

parentCategories = test_set['parent_category_name'].astype('category')
print(len(parentCategories.unique()))
codedParentCategories = parentCategories.cat.rename_categories(np.arange(0, len(parentCategories.unique()))).astype('float')
cleanParentCategories = pd.Series(preprocessing.scale(codedParentCategories))
cleanParentCategories.describe()
print(len(cleanParentCategories.unique()))

clean_price = test_set['price'].fillna(0)
clean_price = preprocessing.scale(clean_price)

userTypes = pd.get_dummies(test_set['user_type'])


306069
306069
28
28
47
47
9
9


In [141]:
df = pd.DataFrame(cleanuserIds, columns=['userIds'])
df['region'] = cleanRegions
df['city'] = cleanCities
df['parent_category_name'] = cleanParentCategories
df['category_name'] = cleanCategories
df['price'] = clean_price
df['Company'] = userTypes['Company']
df['Private'] = userTypes['Private']
df['Shop'] = userTypes['Shop']
X_test = df
X_test.head()

,userIds,region,city,parent_category_name,category_name,price,Company,Private,Shop
0,1.245315,-1.286770,-0.977106,0.085303,-0.990261,-0.050567,0,1,0
1,-1.110267,0.711323,0.802000,2.032842,-1.363694,-0.049990,0,1,0
2,-1.580674,-0.221121,0.751168,-1.862236,-1.587754,-0.047682,0,1,0
3,-0.443540,0.578116,0.039526,-0.888467,-1.438381,-0.049701,0,1,0
4,-1.249306,0.045292,-1.279978,0.085303,1.399710,-0.049624,0,1,0


In [142]:
test_set.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,На ангарском,NaN,66,2017-04-18,Private,a8b57acb5ab304f9c331ac7a074219aed4d349d8aef386...,2020.0
1,65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,NaN,NaN,Продам велосипед,"Продам велосипед KAMA F200,в нормальном состо...",3000.0,4,2017-04-16,Private,NaN,NaN
2,8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,NaN,NaN,BBK,Продам новый телевизор BBK 32 диагональ смарт...,15000.0,15,2017-04-17,Private,8c361112cb049745ef2d1b0ae73594fc5c107286b0c942...,2960.0
3,8e348601fefc,5f1d5c3ce0da,Саратовская область,Саратов,Для дома и дачи,Бытовая техника,Для кухни,Вытяжки,NaN,Вытяжка Jetair 60,"Продам новую вытяжку в упаковке,с документами....",4500.0,70,2017-04-17,Private,NaN,NaN
4,8bd2fe400b89,23e2d97bfc7f,Оренбургская область,Бузулук,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска зима-лето,Продам отличную коляску. б/у 1 год. все вопрос...,4900.0,15,2017-04-15,Private,bc3cf6deef10840fc302e38eb48fa7748aa1e28d534f8f...,1002.0


In [143]:
Y_test = regressor.predict(X_test)

In [144]:
df_test = pd.DataFrame(test_set['item_id'], columns=['item_id'])
df_test['deal_probability'] = Y_test
pd.Series(Y_test).describe()

count    508438.000000
mean          0.160741
std           0.128533
min           0.000000
25%           0.063641
50%           0.127400
75%           0.225384
max           0.980000
dtype: float64

In [145]:
df_test.head()

,item_id,deal_probability
0,6544e41a8817,0.100447
1,65b9484d670f,0.105077
2,8bab230b2ecd,0.206217
3,8e348601fefc,0.200967
4,8bd2fe400b89,0.232826


In [146]:
df_test.to_csv('output.csv', index_label=False, index=False)

In [147]:
test_set.head()


,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,На ангарском,NaN,66,2017-04-18,Private,a8b57acb5ab304f9c331ac7a074219aed4d349d8aef386...,2020.0
1,65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,NaN,NaN,Продам велосипед,"Продам велосипед KAMA F200,в нормальном состо...",3000.0,4,2017-04-16,Private,NaN,NaN
2,8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,NaN,NaN,BBK,Продам новый телевизор BBK 32 диагональ смарт...,15000.0,15,2017-04-17,Private,8c361112cb049745ef2d1b0ae73594fc5c107286b0c942...,2960.0
3,8e348601fefc,5f1d5c3ce0da,Саратовская область,Саратов,Для дома и дачи,Бытовая техника,Для кухни,Вытяжки,NaN,Вытяжка Jetair 60,"Продам новую вытяжку в упаковке,с документами....",4500.0,70,2017-04-17,Private,NaN,NaN
4,8bd2fe400b89,23e2d97bfc7f,Оренбургская область,Бузулук,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска зима-лето,Продам отличную коляску. б/у 1 год. все вопрос...,4900.0,15,2017-04-15,Private,bc3cf6deef10840fc302e38eb48fa7748aa1e28d534f8f...,1002.0


In [148]:
df_test.describe()


,deal_probability
count,508438.000000
mean,0.160741
std,0.128533
min,0.000000
25%,0.063641
50%,0.127400
75%,0.225384
max,0.980000


<h2>Data cleaning</h2>

1. categories: userId, region, city, parent category name, category name
2. one hot encode user type
3. normalize price and other category

In [1]:
regions = train_set['region'].astype('category')
print(len(regions.unique()))
codedRegions = regions.cat.rename_categories(np.arange(0, len(regions.unique()))).astype('float')
cleanRegions = pd.Series(preprocessing.scale(codedRegions))
cleanRegions.describe()
print(len(cleanRegions.unique()))

categories = train_set['category_name'].astype('category')
print(len(categories.unique()))
codedCategories = categories.cat.rename_categories(np.arange(0, len(categories.unique()))).astype('float')
cleanCategories = pd.Series(preprocessing.scale(codedCategories))
cleanCategories.describe()
print(len(cleanCategories.unique()))

parentCategories = train_set['parent_category_name'].astype('category')
print(len(parentCategories.unique()))
codedParentCategories = parentCategories.cat.rename_categories(np.arange(0, len(parentCategories.unique()))).astype('float')
cleanParentCategories = pd.Series(preprocessing.scale(codedParentCategories))
cleanParentCategories.describe()
print(len(cleanParentCategories.unique()))

userIds = train_set['user_id'].astype('category')
print(len(userIds.unique()))
codeduserIds = userIds.cat.rename_categories(np.arange(0, len(userIds.unique()))).astype('float')
cleanuserIds = pd.Series(preprocessing.scale(codeduserIds))
cleanuserIds.describe()
print(len(cleanuserIds.unique()))

cities = train_set['city'].astype('category')
print(len(cities.unique()))
codedCities = cities.cat.rename_categories(np.arange(0, len(cities.unique()))).astype('float')
cleanCities = pd.Series(preprocessing.scale(codedCities))
cleanCities.describe()
len(cleanCities.unique())

userTypes = pd.get_dummies(train_set['user_type'])


clean_price = train_set['price'].fillna(0)
clean_price = preprocessing.scale(clean_price)
clean_price.mean()

df = pd.DataFrame(cleanuserIds, columns=['userIds'])
df['region'] = cleanRegions
df['city'] = cleanCities
df['parent_category_name'] = cleanParentCategories
df['category_name'] = cleanCategories
df['price'] = clean_price
df['Company'] = userTypes['Company']
df['Private'] = userTypes['Private']
df['Shop'] = userTypes['Shop']
Y = train_set['deal_probability']
X = df
X.head()

NameError: name 'train_set' is not defined

Data cleaning

1. categories: userId, region, city, parent category name, category name
2. one hot encode user type, parent_category_name
3. normalize only price


In [33]:
regions = train_set['region'].astype('category')
print(len(regions.unique()))
codedRegions = regions.cat.rename_categories(np.arange(0, len(regions.unique()))).astype('float')
cleanRegions = codedRegions
cleanRegions.describe()
print(len(cleanRegions.unique()))

categories = train_set['category_name'].astype('category')
print(len(categories.unique()))
codedCategories = categories.cat.rename_categories(np.arange(0, len(categories.unique()))).astype('float')
cleanCategories = codedCategories
cleanCategories.describe()
print(len(cleanCategories.unique()))

parent_category_names = pd.get_dummies(train_set['parent_category_name'])

userIds = train_set['user_id'].astype('category')
print(len(userIds.unique()))
codeduserIds = userIds.cat.rename_categories(np.arange(0, len(userIds.unique()))).astype('float')
cleanuserIds = codeduserIds
cleanuserIds.describe()
print(len(cleanuserIds.unique()))

cities = train_set['city'].astype('category')
print(len(cities.unique()))
codedCities = cities.cat.rename_categories(np.arange(0, len(cities.unique()))).astype('float')
cleanCities = codedCities
cleanCities.describe()
len(cleanCities.unique())

userTypes = pd.get_dummies(train_set['user_type'])


clean_price = train_set['price'].fillna(0)
clean_price = preprocessing.scale(clean_price)
clean_price.mean()



28
28
47
47
771769
771769
1733


2.776620948309052e-19

In [34]:
df = pd.DataFrame()
df['userIds'] = cleanuserIds
df['region'] = cleanRegions
df['city'] = cleanCities
#df['parent_category_name'] = cleanParentCategories
df['category_name'] = cleanCategories
df['price'] = clean_price
df['Company'] = userTypes['Company']
df['Private'] = userTypes['Private']
df['Shop'] = userTypes['Shop']
#df['item_seq_number'] = train_set['item_seq_number']

#Y = train_set['deal_probability']
#X = df
#X.head()

In [35]:
for cat in list(parent_category_names):
    df[cat]=parent_category_names[cat]

In [36]:
Y = train_set['deal_probability']
X = df
X.head()

,userIds,region,city,category_name,price,Company,Private,Shop,Бытовая электроника,Для бизнеса,Для дома и дачи,Животные,Личные вещи,Недвижимость,Транспорт,Услуги,Хобби и отдых
0,675853.0,19.0,460.0,42.0,-0.004592,0,1,0,0,0,0,0,1,0,0,0,0
1,173962.0,17.0,1300.0,22.0,-0.004552,0,1,0,0,0,1,0,0,0,0,0,0
2,440069.0,16.0,1276.0,2.0,-0.004537,0,1,0,1,0,0,0,0,0,0,0,0
3,576929.0,21.0,940.0,42.0,-0.004564,1,0,0,0,0,0,0,1,0,0,0,0
4,721853.0,4.0,317.0,0.0,-0.003983,0,1,0,0,0,0,0,0,0,1,0,0


In [10]:


regressor2 = RandomForestRegressor(n_jobs=10,n_estimators=10)
scores2 = cross_val_score(regressor2, X, Y, cv=5)
print(scores2.mean())
print(scores2.std())

0.05685850802720862
0.0026841214901724285


In [17]:


regressor3= RandomForestRegressor(n_jobs=10,n_estimators=10)
scores3 = cross_val_score(regressor3, X, Y, cv=5)


0.05685850802720862
0.0026841214901724285


In [18]:
print(scores3.mean())
print(scores3.std())

0.01155981055229618
0.002556555673566709


In [38]:
regions = test_set['region'].astype('category')
print(len(regions.unique()))
codedRegions = regions.cat.rename_categories(np.arange(0, len(regions.unique()))).astype('float')
cleanRegions = codedRegions
cleanRegions.describe()
print(len(cleanRegions.unique()))

categories = test_set['category_name'].astype('category')
print(len(categories.unique()))
codedCategories = categories.cat.rename_categories(np.arange(0, len(categories.unique()))).astype('float')
cleanCategories = codedCategories
cleanCategories.describe()
print(len(cleanCategories.unique()))

parent_category_names = pd.get_dummies(test_set['parent_category_name'])

userIds = test_set['user_id'].astype('category')
print(len(userIds.unique()))
codeduserIds = userIds.cat.rename_categories(np.arange(0, len(userIds.unique()))).astype('float')
cleanuserIds = codeduserIds
cleanuserIds.describe()
print(len(cleanuserIds.unique()))

cities = test_set['city'].astype('category')
print(len(cities.unique()))
codedCities = cities.cat.rename_categories(np.arange(0, len(cities.unique()))).astype('float')
cleanCities = codedCities
cleanCities.describe()
len(cleanCities.unique())

userTypes = pd.get_dummies(test_set['user_type'])


clean_price = test_set['price'].fillna(0)
clean_price = preprocessing.scale(clean_price)
clean_price.mean()



28
28
47
47
306069
306069
1644


4.016069190128566e-18

In [39]:
df = pd.DataFrame()
df['userIds'] = cleanuserIds
df['region'] = cleanRegions
df['city'] = cleanCities
#df['parent_category_name'] = cleanParentCategories
df['category_name'] = cleanCategories
df['price'] = clean_price
df['Company'] = userTypes['Company']
df['Private'] = userTypes['Private']
df['Shop'] = userTypes['Shop']
#df['item_seq_number'] = train_set['item_seq_number']

#Y = train_set['deal_probability']
#X = df
#X.head()

for cat in list(parent_category_names):
    df[cat]=parent_category_names[cat]
    
X_test = df
X_test.head()

,userIds,region,city,category_name,price,Company,Private,Shop,Бытовая электроника,Для бизнеса,Для дома и дачи,Животные,Личные вещи,Недвижимость,Транспорт,Услуги,Хобби и отдых
0,262938.0,4.0,306.0,10.0,-0.050567,0,1,0,0,0,0,0,1,0,0,0,0
1,55145.0,19.0,933.0,5.0,-0.049990,0,1,0,0,0,0,0,0,0,0,0,1
2,13649.0,12.0,147.0,2.0,-0.047682,0,1,0,1,0,0,0,0,0,0,0,0
3,113959.0,18.0,1238.0,4.0,-0.049701,0,1,0,0,0,1,0,0,0,0,0,0
4,42880.0,14.0,236.0,42.0,-0.049624,0,1,0,0,0,0,0,1,0,0,0,0


In [27]:
from sklearn.model_selection import cross_val_predict
y_pred = regressor3.predict(X)

NotFittedError: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [55]:
X.shape

(1503424, 17)

In [56]:
train_set.shape

(1503424, 18)

In [ ]:
regressor4 = RandomForestRegressor(n_jobs=50,n_estimators=50)
regressor4.fit(X, Y)

In [ ]:
y_pred = regressor4.predict(X_test)

In [ ]:
df_test = pd.DataFrame()
df_test['item_id'] = test_set['item_id']
df_test['deal_probability'] = y_pred

In [ ]:
df_test.head()


In [49]:
df_test.describe()

,deal_probability
count,508438.000000
mean,0.161233
std,0.146316
min,0.000000
25%,0.048457
50%,0.119911
75%,0.235475
max,1.000000


In [50]:
df_test.to_csv('output2.csv', index=False, index_label=False)